In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import bdsf
import mosaic as mosaic # a python function that reads images and weights, and gives optimally weighted mosaic.

In [ ]:
input_images = ["AAAA.SP2B.PBCOR.SMOOTH.FITS", "BBBB.SP2B.PBCOR.SMOOTH.FITS", "CCCC.SP2B.PBCOR.SMOOTH.FITS", "DDDD.SP2B.PBCOR.SMOOTH.FITS", 
                "EEEE.SP2B.PBCOR.SMOOTH.FITS", "FFFF.SP2B.PBCOR.SMOOTH.FITS", "GGGG.SP2B.PBCOR.SMOOTH.FITS"]
# input_images = ["AAAA.SP2B.PBCOR.FITS", "BBBB.SP2B.PBCOR.FITS", "CCCC.SP2B.PBCOR.FITS", "DDDD.SP2B.PBCOR.FITS", 
#                 "EEEE.SP2B.PBCOR.FITS", "FFFF.SP2B.PBCOR.FITS", "GGGG.SP2B.PBCOR.FITS"]

weights = ["AAAA.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", "BBBB.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", "CCCC.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", "DDDD.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", 
           "EEEE.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", "FFFF.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits", "GGGG.SP2B.PBCOR.SMOOTH.pybdsf_rms.fits"]
# weights = ["AAAA.SP2B.PBCOR.pybdsf_rms.fits", "BBBB.SP2B.PBCOR.pybdsf_rms.fits", "CCCC.SP2B.PBCOR.pybdsf_rms.fits", "DDDD.SP2B.PBCOR.pybdsf_rms.fits", 
#            "EEEE.SP2B.PBCOR.pybdsf_rms.fits", "FFFF.SP2B.PBCOR.pybdsf_rms.fits", "GGGG.SP2B.PBCOR.pybdsf_rms.fits"]


In [ ]:
    # img = bdsf.process_image(input_images[ii], rms_box=(200, 20), adaptive_rms_box=True, 
    #                          adaptive_thresh=100, rms_box_bright=(80, 20), clobber=True, quiet=True)

In [ ]:
for ii in range(len(input_images)): # 
    img = bdsf.process_image(input_images[ii], psf_vary_do = True, adaptive_rms_box = True, 
                             rms_box=(150, 30), rms_box_bright=(50, 10), clobber=True, quiet=True)
    img.export_image(img_type='rms', clobber=True)



In [ ]:
# call the mosaic function
mosaic.mosaic_fits(input_images, weights)

In [ ]:
img = bdsf.process_image('final_mosaic.fits', psf_vary_do = True, adaptive_rms_box = True, 
                         rms_box=(150, 30), rms_box_bright=(50, 10), clobber=True, quiet=True)

img.write_catalog(format='fits', catalog_type='srl', clobber=True)
img.write_catalog(format='ascii', catalog_type='gaul', clobber=True)

img.export_image(img_type='rms', outfile='rms_map.fits', clobber=True)
img.export_image(img_type='gaus_resid', outfile='gaus_resid.fits', clobber=True)
img.export_image(img_type='gaus_model', outfile='gaus_model.fits', clobber=True)



In [ ]:
# Read back the RMS image
from astropy.io import fits
with fits.open("final_mosaic.pybdsf_rms.fits") as hdul:
    rms_map = hdul[0].data  # 2D array of RMS values

global_rms = np.nanmedian(rms_map)  # Compute mean RMS over the image

print(f"Off-source RMS (PyBDSF): {global_rms:.6f} Jy/beam")

In [ ]:
# Read back the RMS image
from astropy.io import fits

for fields in weights:
    with fits.open(fields) as hdul:
        rms_map = hdul[0].data  # 2D array of RMS values
    
    global_rms = np.nanmedian(rms_map)  # Compute mean RMS over the image
    
    print(f"Off-source RMS (PyBDSF): {global_rms:.6f} Jy/beam")

In [ ]:
# img_type ....... 'gaus_resid': Type of image to export: 'gaus_resid',
#                                'shap_resid', 'rms', 'mean', 'gaus_model',
#                                'shap_model', 'ch0', 'pi', 'psf_major', 'psf_minor',
#                                'psf_pa', 'psf_ratio', 'psf_ratio_aper', 'island_mask'

In [ ]:
import numpy as np
from astropy.io import fits

# Read the RMS image
with fits.open("final_mosaic.pybdsf_rms.fits") as hdul:
    rms_map = hdul[0].data  # 2D array of RMS values

# Compute percentiles and median
median_rms = np.nanmedian(rms_map)
p16 = np.nanpercentile(rms_map, 16)
p84 = np.nanpercentile(rms_map, 84)

# Compute symmetric error bounds
lower_bound = median_rms - p16
upper_bound = p84 - median_rms

# Print results
print(f"Off-source RMS (median, PyBDSF): {median_rms:.6f} Jy/beam")
print(f"Error bounds: -{lower_bound:.6f} / +{upper_bound:.6f} Jy/beam")
